# BOExplorer
### Diary database creation

Supported character sets and collations in __MariaDB__:
https://mariadb.com/kb/en/supported-character-sets-and-collations/

#### Connection parameters:

In [1]:
db_host = 'localhost'
db_database = 'boe'
db_user = 'root'
db_password = 'pass'

#### 1. Imports

In [2]:
from typing import Callable, Set, Generator, Iterable, Dict
import datetime
import collections
import functools

import pymysql

from helpers import helpers
from helpers import boe

#### 2. Database tables

In [3]:
boe_diary_section_table = '''
CREATE TABLE boe_diary_section (
    id VARCHAR(2) NOT NULL,
    name VARCHAR(60) NOT NULL,
    PRIMARY KEY (id)
) character set=utf8;
'''

boe_diary_entry_label_table = '''
CREATE TABLE boe_diary_entry_label (
    name VARCHAR(500),
    entry VARCHAR(20) NOT NULL,
    FOREIGN KEY (entry)
        REFERENCES boe_diary_entry (id)
        ON DELETE CASCADE
) character set=utf8;
'''

boe_diary_entry_references_table = '''
CREATE TABLE boe_diary_entry_reference (
    source VARCHAR(20) NOT NULL,
    referenced VARCHAR(20) NOT NULL,
    FOREIGN KEY (source)
        REFERENCES boe_diary_entry (id)
        ON DELETE CASCADE,
    FOREIGN KEY (referenced)
        REFERENCES boe_diary_entry (id)
        ON DELETE CASCADE
) character set=utf8;
'''

boe_diary_entry_table = '''
CREATE TABLE boe_diary_entry (
    id VARCHAR(20) NOT NULL,
    date DATE,
    title VARCHAR(1000),
    section VARCHAR(2) NOT NULL,
    department VARCHAR(500),
    epigraph VARCHAR(1000),
    pdf_url VARCHAR(240),
    xml_url VARCHAR(240),
    htm_url VARCHAR(240),
    has_economic_impact VARCHAR(20),
    PRIMARY KEY (id),
    FOREIGN KEY (section)
        REFERENCES boe_diary_section (id)
        ON DELETE CASCADE
) character set=utf8;
'''

#### 3. Helper functions

In [4]:
def create_db_settings_wrapper(db_host, db_database, db_user, db_password):
    def caller(f, *args, **kwargs):
        keyword_args = dict(kwargs)
        keyword_args.update({
            'host': db_host,
            'database': db_database,
            'user': db_user,
            'password': db_password
        })
        return f(*args, **keyword_args)
    return caller

def object_exists(cursor: pymysql.cursors.Cursor, query: str, object_name: str) -> bool:
    '''Check if an SQL object is present in the server.'''
    
    cursor.execute(query)
    objects = cursor.fetchall()
    exists = helpers.pipe(objects,
        functools.partial(map, lambda x: x[0]),
        lambda x: object_name in x)
    
    return exists

def database_exists(cursor: pymysql.cursors.Cursor, db_name:str) -> bool:
    '''Check if the database is present in the SQL sever.'''
    
    return object_exists(cursor, 'show databases', db_name)

def table_exists(cursor: pymysql.cursors.Cursor, table_name:str) -> bool:
    '''Check if the table is present in the given database.'''
    
    return object_exists(cursor, 'show tables', table_name)

def create_database(host: str, user: str, password: str, database: str, force=False) -> None:
    '''Create the given database.'''
    
    connection = pymysql.connect(host=host,
                                 user=user,
                                 password=password)

    with connection:
        with connection.cursor() as cursor:
            if not database_exists(cursor, database):
                cursor.execute(f'create database {database}')
            elif force is True:
                cursor.execute(f'drop database {database}')
                cursor.execute(f'create database {database}')
        connection.commit()
            
def create_tables(host, user, password, database, tables) -> None:
    '''Create the given tables provided as name, query arguments.
    >>> create_tables(db_host, 
              db_user, 
              db_password,
              db_database,
              'boe_diary_section',
              boe_diary_section_table,
              'boe_diary_entry',
              boe_diary_entry_table)
    '''
    
    connection = pymysql.connect(
        host=host,
        user=user,
        password=password,
        database=database)

    with connection:
        with connection.cursor() as cursor:
            for table_name, table_query in zip(tables[::2], tables[1::2]):
                if not table_exists(cursor, table_name):
                    cursor.execute(table_query)
        connection.commit()
                
def insert_sections(host, user, password, database, sections: Dict) -> None:
    '''Insert the information for the different BOE sections.'''
    
    connection = pymysql.connect(
        host=host,
        user=user,
        password=password,
        database=database)

    with connection:
        with connection.cursor() as cursor:
            if table_exists(cursor, 'boe_diary_section'):
                for section_id, section_name in sections.items():
                    query = f"INSERT INTO boe_diary_section VALUES ('{section_id}', '{section_name}');"
                    cursor.execute(query)

        connection.commit()

#### 4. Execution

In [5]:
call_with_db_settings = \
    create_db_settings_wrapper(db_host, db_database, db_user, db_password)

call_with_db_settings(create_database, force=True)

call_with_db_settings(create_tables, tables=(
    'boe_diary_section',
    boe_diary_section_table,
    'boe_diary_entry',
    boe_diary_entry_table,
    'boe_diary_label',
    boe_diary_entry_label_table,
    'boe_diary_entry_references',
    boe_diary_entry_references_table))

call_with_db_settings(insert_sections, sections=boe.code_to_section_name)